<a href="https://colab.research.google.com/github/NamPham1010/Machine_Learning_SPY/blob/main/ML_Project_MA_SPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [2]:
# Downloading SPY historical price data
spy = yf.Ticker("SPY")
spy_price = yf.download("SPY", start = "2015-01-01", end = "2022-12-31")

[*********************100%%**********************]  1 of 1 completed


In [3]:
# Loading data into dataframe
df = pd.DataFrame(spy_price)
df.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,206.380005,206.880005,204.179993,205.429993,175.515549,121465900
2015-01-05,204.169998,204.369995,201.350006,201.720001,172.345779,169632600
2015-01-06,202.089996,202.720001,198.860001,199.820007,170.722427,209151400
2015-01-07,201.419998,202.720001,200.880005,202.309998,172.849854,125346700
2015-01-08,204.009995,206.160004,203.990005,205.899994,175.917099,147217800
2015-01-09,206.399994,206.419998,203.509995,204.250000,174.507370,158567300
2015-01-12,204.410004,204.600006,201.919998,202.649994,173.140305,144396100
2015-01-13,204.119995,205.479996,200.509995,202.080002,172.653336,214553300
2015-01-14,199.649994,201.100006,198.570007,200.860001,171.610992,192991100


In [4]:
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-12-16,385.179993,386.579987,381.040009,383.269989,379.062683,119858000
2022-12-19,383.470001,383.820007,378.279999,380.019989,375.848389,79878100
2022-12-20,379.230011,382.230011,377.850006,380.540009,376.362671,74427200
2022-12-21,383.250000,387.410004,382.690002,386.230011,381.990204,78167400
2022-12-22,383.049988,386.209991,374.769989,380.720001,376.540649,100120900
2022-12-23,379.649994,383.059998,378.029999,382.910004,378.706665,59857300
2022-12-27,382.790009,383.149994,379.649994,381.399994,377.213226,51638200
2022-12-28,381.329987,383.390015,376.420013,376.660004,372.525238,70911500
2022-12-29,379.630005,384.350006,379.079987,383.440002,379.230835,66970900


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2014 entries, 2015-01-02 to 2022-12-30
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2014 non-null   float64
 1   High       2014 non-null   float64
 2   Low        2014 non-null   float64
 3   Close      2014 non-null   float64
 4   Adj Close  2014 non-null   float64
 5   Volume     2014 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 110.1 KB


In [6]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,2014.000000,2014.000000,2014.000000,2014.000000,2014.000000,2.014000e+03
mean,297.732294,299.417815,295.867716,297.755636,277.662367,9.187613e+07
std,80.871703,81.471799,80.177458,80.853811,86.456464,4.765484e+07
min,182.339996,184.100006,181.020004,182.860001,159.456055,2.027000e+07
25%,226.322498,226.757500,225.507496,226.255001,201.462524,6.037552e+07
50%,279.395004,280.580002,277.994995,279.330002,257.639542,7.905045e+07
75%,367.944992,370.119995,364.459999,366.842506,354.429993,1.080564e+08
max,479.220001,479.980011,476.059998,477.709991,464.923370,5.072443e+08


**Data Wrangling**

In [10]:
df.duplicated()

Date
2015-01-02    False
2015-01-05    False
2015-01-06    False
2015-01-07    False
2015-01-08    False
              ...  
2022-12-23    False
2022-12-27    False
2022-12-28    False
2022-12-29    False
2022-12-30    False
Length: 2014, dtype: bool